<a href="https://colab.research.google.com/github/GuilhermeOchoa/Data-Science/blob/master/Jogo_da_Velha_entre_IA's_20241109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Debugger:
    def __init__(self, enabled=False):
        self.enabled = enabled

    def set_debug(self, enabled):
        """Habilita ou desabilita a impressão de variáveis de depuração."""
        self.enabled = enabled

    def debug_print(self, *args, **kwargs):
        """Imprime apenas se o modo de depuração estiver habilitado."""
        if self.enabled:
            print(*args, **kwargs)


In [ ]:
import numpy as np
import random

class Sucessor:
    def __init__(self, estado, valor, coluna=None):
        self.estado = estado
        self.valor = valor
        self.coluna = coluna

    def get_estado(self):
        return self.estado

    def get_valor(self):
        return self.valor

    def get_coluna(self):
        return self.coluna

    def __str__(self):
        return f"Estado: {self.estado}, Valor: {self.valor}, Coluna: {self.coluna}"

class Minimax:
    def __init__(self, estado):
        self.estado = estado

    def get_melhor(self):
        melhor_sucessor = self.algoritmo(self.estado, False, self.livres(self.estado))
        return melhor_sucessor.coluna if melhor_sucessor else None  # Retorna o índice da melhor jogada

    def get_melhor_ab(self):
        melhor_sucessor = self.algoritmo_ab(self.estado, False, self.livres(self.estado), -999, 999)
        return melhor_sucessor.coluna if melhor_sucessor else None  # Retorna o índice da melhor jogada

    def livres(self, estado):
        return estado.count('0')

    def jogada_aleatoria(self):
        posicoes_livres = [i for i, v in enumerate(self.estado) if v == '0']
        return random.choice(posicoes_livres) if posicoes_livres else None

    def gera_vizinhos(self, estado, caracter):
        vizinhos = []
        for i in range(9):
            if estado[i] == '0':
                novo_estado = estado[:]
                novo_estado[i] = caracter
                vizinhos.append((novo_estado, i))  # Retorna o estado e o índice da jogada
        return vizinhos

    def utilidade(self, atual, profundidade):
        if self.vencedor(atual, 'X'):
            return -1
        if self.vencedor(atual, 'O'):
            return 1
        if profundidade == 0:
            return 0
        return 100

    def vencedor(self, atual, caracter):
        # Verifica linhas, colunas e diagonais para vitória
        for i in range(3):
            if all(atual[i*3 + j] == caracter for j in range(3)) or all(atual[i + j*3] == caracter for j in range(3)):
                return True
        return (atual[0] == caracter and atual[4] == caracter and atual[8] == caracter) or \
               (atual[2] == caracter and atual[4] == caracter and atual[6] == caracter)

    def algoritmo(self, estado, jogador, profundidade):
        valor = self.utilidade(estado, profundidade)
        if valor != 100:
            return Sucessor(estado, valor)

        vizinhos = self.gera_vizinhos(estado, 'X' if jogador else 'O')
        melhor_sucessor = None
        if jogador:
            menor = 999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo(vizinho, False, profundidade - 1)
                if atual.get_valor() < menor:
                    menor = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, menor, coluna=indice)  # Armazena o índice
            return melhor_sucessor
        else:
            maior = -999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo(vizinho, True, profundidade - 1)
                if atual.get_valor() > maior:
                    maior = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, maior, coluna=indice)  # Armazena o índice
            return melhor_sucessor

    def algoritmo_ab(self, estado, jogador, profundidade, alfa, beta):
        valor = self.utilidade(estado, profundidade)
        if valor != 100:
            return Sucessor(estado, valor)

        vizinhos = self.gera_vizinhos(estado, 'X' if jogador else 'O')
        melhor_sucessor = None
        if jogador:
            menor = 999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo_ab(vizinho, False, profundidade - 1, alfa, beta)
                if atual.get_valor() < menor:
                    menor = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, menor, coluna=indice)  # Armazena o índice
                if menor < alfa:
                    return melhor_sucessor
                beta = min(beta, menor)
            return melhor_sucessor
        else:
            maior = -999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo_ab(vizinho, True, profundidade - 1, alfa, beta)
                if atual.get_valor() > maior:
                    maior = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, maior, coluna=indice)  # Armazena o índice
                if maior > beta:
                    return melhor_sucessor
                alfa = max(alfa, maior)
            return melhor_sucessor


In [ ]:
# Definição da Rede Neural
# Função de ativação
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Definição da Rede Neural
class MLP:
    def __init__(self, input_size, hidden_size, output_size, population_size, generations, mutation_rate, debugger):
        # Instancia o algoritmo genético para gerar os pesos iniciais
        self.genetic_algorithm = GeneticAlgorithm(input_size, hidden_size, output_size, population_size, generations, mutation_rate, debugger)
        self.population_size = population_size
        # Obtém a população inicial de cromossomos do algoritmo genético
        self.population = self.genetic_algorithm.initialize_population()
        self.current_chromosome_index = 0  # Índice do cromossomo atual
        self.debugger = debugger

    def set_current_chromosome(self):
        # Define os pesos da camada oculta e de saída com base no cromossomo atual
        self.hidden_weights, self.output_weights = self.population[self.current_chromosome_index]
        self.debugger.debug_print(f"Pesos da camada oculta do cromossomo {self.current_chromosome_index} : {self.hidden_weights}")
        self.debugger.debug_print(f"Pesos da camada de saída do cromossomo {self.current_chromosome_index} : {self.output_weights}")

    def next_chromosome(self):
        # Avança para o próximo cromossomo na população
        self.current_chromosome_index = (self.current_chromosome_index + 1) % self.population_size
        self.set_current_chromosome()
        if self.current_chromosome_index == 0:
            self.debugger.debug_print("Todos os cromossomos foram avaliados. Recalibrando pesos...")
            self.refinar_pesos()
            #self.debugger.debug_print("Pesos atualizados da camada oculta após o refinamento:", self.hidden_weights)
            #self.debugger.debug_print("Pesos atualizados da camada de saída após o refinamento:", self.output_weights)



    def refinar_pesos(self):
            # Recalibra os pesos quando todos os cromossomos tiverem sido avaliados
            self.hidden_weights, self.output_weights = self.genetic_algorithm.refinar_pesos()

    def update_fitness(self, game_result, invalid_moves):
        # Atualiza o fitness do cromossomo atual com base no resultado do jogo
        self.genetic_algorithm.set_fitness(self.current_chromosome_index, game_result, invalid_moves)
        updated_fitness = self.genetic_algorithm.get_fitness(self.current_chromosome_index)
       # print(f"Atualizado o fitness do cromossomo {self.current_chromosome_index} para: {updated_fitness}")

    # Passo de inferência para escolher a jogada
    def forward(self, inputs):
        inputs = np.append(inputs, 1)  # Adiciona bias
        self.debugger.debug_print("Entradas da rede neural: ",inputs)
        hidden_inputs = np.dot(self.hidden_weights, inputs)
        hidden_outputs = sigmoid(hidden_inputs)
        #self.debugger.debug_print("Pesos * sigmoid  nas Saídas da camada oculta: ",hidden_outputs)
        hidden_outputs = np.append(hidden_outputs, 1)  # Adiciona bias
        final_inputs = np.dot(self.output_weights, hidden_outputs)
        final_outputs = sigmoid(final_inputs)
        return final_outputs

    # Escolhe a jogada baseada na saída da rede neural
    def escolher_jogada(self, tabuleiro):
        inputs = np.arrayinputs =np.array([1 if pos == 'X' else -1 if pos == 'O' else 0 for pos in tabuleiro])
        outputs = self.forward(inputs)
        self.debugger.debug_print("Saídas da rede neural: ",outputs)
        melhor_jogada = np.argmax(outputs)
        return melhor_jogada



In [ ]:
class GeneticAlgorithm:
    def __init__(self, input_size, hidden_size, output_size, population_size, generations, mutation_rate, debugger=None):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.population = self.initialize_population()
        self.fitness_scores = [0] * self.population_size  # Fitness inicial de cada cromossomo
        self.debugger = debugger

    debugger = Debugger()
     # Inicializa a população com pesos randômicos
    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            hidden_weights = np.random.uniform(-1, 1, (self.hidden_size, self.input_size + 1))
            output_weights = np.random.uniform(-1, 1, (self.output_size, self.hidden_size + 1))
            population.append((hidden_weights, output_weights))
        return population

    # Função de aptidão
    def fitness_function(self, game_result, invalid_moves):
        score = 0
        if game_result == 1:
            score += 1000
        elif game_result == 0:
            score += 5
        elif game_result == -1:
            score -= 5
        score -= invalid_moves * 2
        return score

    def set_fitness(self, chromosome_index, game_result, invalid_moves):
        score = self.fitness_function(game_result, invalid_moves)
        self.fitness_scores[chromosome_index] += score
        print(f"Fitness atualizado para cromossomo {chromosome_index}: {self.fitness_scores[chromosome_index]}")

    def get_fitness(self, chromosome_index):
        return self.fitness_scores[chromosome_index]

    # Seleciona os melhores indivíduos da população atual
    def selecionar(self):
        sorted_indices = np.argsort(self.fitness_scores)[::-1]
        sorted_population = [self.population[i] for i in sorted_indices]
        self.debugger.debug_print("População ordenada:", sorted_population)
        return sorted_population

    import numpy as np

    def crossover(self, parent1, parent2):
     # Calcula o vetor médio para os pesos escondidos e de saída
      child_hidden = (parent1[0] + parent2[0]) / 2
      child_output = (parent1[1] + parent2[1]) / 2

      return (child_hidden, child_output)

    # Mutação com baixa taxa em genes selecionados aleatoriamente
    def mutate(self, weight_matrix):
        mutation_mask = np.random.rand(*weight_matrix.shape) < self.mutation_rate
        weight_matrix += mutation_mask * np.random.uniform(-0.5, 0.5, weight_matrix.shape)

    # Método principal para refinar pesos
    def refinar_pesos(self):
        for generation in range(self.generations):
            sorted_population = self.selecionar()
            # Critério de parada por convergência
            if np.std(self.fitness_scores) / np.mean(self.fitness_scores) < 0.01:  # Convergência de 98 a 99%
                break

            # Construção da nova população
            new_population = sorted_population[:self.population_size // 10]  # Elite
            while len(new_population) < self.population_size:
                parent1, parent2 = random.choice(sorted_population), random.choice(sorted_population)
                child1, child2 = self.crossover(parent1, parent2)

                # Aplicação de mutação em genes dos filhos
                if random.random() < 0.5:
                    self.mutate(child1[0])
                    self.mutate(child1[1])
                else:
                    self.mutate(child2[0])
                    self.mutate(child2[1])

                new_population.extend([child1, child2])

            self.population = new_population[:self.population_size]

           # self.debugger.debug_print("Nova população:", self.population)
        print(f"Geração {generation + 1} concluída. Nova população: {self.population}")
        best_hidden, best_output = self.population[np.argmax(self.fitness_scores)]
        #self.debugger.debug_print("Melhor conjunto de pesos:", best_hidden, best_output)
        return best_hidden, best_output



In [ ]:
2# Laço principal com o jogo
import random
def inicializar_tabuleiro():
    return ['0' for _ in range(9)]

def imprimir_tabuleiro(tabuleiro):
    print(f"{tabuleiro[0]} | {tabuleiro[1]} | {tabuleiro[2]}")
    print("-" * 10)
    print(f"{tabuleiro[3]} | {tabuleiro[4]} | {tabuleiro[5]}")
    print("-" * 10)
    print(f"{tabuleiro[6]} | {tabuleiro[7]} | {tabuleiro[8]}")

def verificar_estado(tabuleiro):
    # Verifica as linhas
    for i in range(0, 9, 3):
        if tabuleiro[i] == tabuleiro[i+1] == tabuleiro[i+2] and tabuleiro[i] != '0':
            return 2 if tabuleiro[i] == 'X' else 1  # 2 para MLP (X), 1 para Minimax (O)

    # Verifica as colunas
    for i in range(3):
        if tabuleiro[i] == tabuleiro[i+3] == tabuleiro[i+6] and tabuleiro[i] != '0':
            return 2 if tabuleiro[i] == 'X' else 1  # 1 para MLP (X), 2 para Minimax (O)

    # Verifica as diagonais
    if tabuleiro[0] == tabuleiro[4] == tabuleiro[8] and tabuleiro[0] != '0':
        return 2 if tabuleiro[0] == 'X' else 1
    if tabuleiro[2] == tabuleiro[4] == tabuleiro[6] and tabuleiro[2] != '0':
        return 2 if tabuleiro[2] == 'X' else 1

    # Verifica se houve empate
    if all(x != '0' for x in tabuleiro):  # Corrige '0' para string
        return 3  # Empate

    return 0  # Jogo continua

def jogada_ia_MiniMax(tabuleiro, dificuldade):
    jogo_ia = Minimax(tabuleiro)

    if dificuldade == 1:  # Nível 1: 25% Minimax, 75% Aleatório
        probabilidade_minimax = 0.25
    elif dificuldade == 2:  # Nível 2: 50% Minimax, 50% Aleatório
        probabilidade_minimax = 0.5
    else:  # Nível 3: 100% Minimax
        probabilidade_minimax = 1.0

    if random.random() < probabilidade_minimax:
        movimento_ia = jogo_ia.get_melhor()  # Pega o índice da jogada
    else:
        movimento_ia = jogo_ia.jogada_aleatoria()

    tabuleiro[movimento_ia] = 'O'
    print(f"IA MiniMax escolheu a posição {movimento_ia}")

def jogada_ia_MLP(tabuleiro, mlp):
    movimento_iaMLP = mlp.escolher_jogada(tabuleiro)
    invalid_move = False

    if tabuleiro[movimento_iaMLP] != '0':  # Verifica se a jogada é inválida
        print(f"Jogada inválida pela IA MLP na posição {movimento_iaMLP}. Penalizando...")
        invalid_move = True
    else:
        tabuleiro[movimento_iaMLP] = 'X'
        print(f"IA MLP escolheu a posição {movimento_iaMLP}")

    return invalid_move  # Retorna se a jogada foi inválida ou não

def jogada_humana(tabuleiro):
    while True:
        try:
            posicao = int(input("Escolha uma posição de 0 a 8: "))
            if tabuleiro[posicao] == '0' and 0 <= posicao <= 8:
                tabuleiro[posicao] = 'X'
                break
            else:
                print("Posição inválida. Tente novamente.")
        except ValueError:
            print("Entrada inválida. Por favor, insira um número de 0 a 8.")



def jogar():
    continuar_jogando = True
    debugger = Debugger(enabled=False)  # Habilita/desabilita o debugger
    mlp = MLP(input_size=9, hidden_size=9, output_size=9, population_size=1000, generations=100, mutation_rate=0.1, debugger=debugger)  # Cria instância da MLP a ser treinada

    while continuar_jogando:
        print("Escolha uma opção de jogo:")
        print("1 - Jogar contra Minimax")
        print("2 - Treinar a rede neural jogando contra Minimax")
        print("3 - Jogar contra a rede neural treinada")

        try:
            escolha = int(input("Digite o número da opção escolhida: "))
            if escolha not in [1, 2, 3]:
                print("Escolha inválida. Tente novamente.")
                continue
        except ValueError:
            print("Escolha inválida. Tente novamente.")
            continue

        num_partidas = int(input("Quantas partidas você deseja jogar? "))
        dificuldade = int(input("Escolha a dificuldade (1: Fácil, 2: Médio, 3: Difícil): "))

        # Contadores para vitórias e empates
        vitorias_jogador = 0
        vitorias_minimax = 0
        vitorias_mlp = 0
        empates = 0
        invalid_moves = 0


        for partida in range(num_partidas):
            print(f"\n--- Partida {partida + 1} ---")
            tabuleiro = inicializar_tabuleiro()
            jogadas = 0
            mlp.set_current_chromosome()
            game_result = 0

            while True:
                imprimir_tabuleiro(tabuleiro)

                if escolha == 1:
                    if jogadas % 2 == 0:
                        jogada_humana(tabuleiro)
                    else:
                        jogada_ia_MiniMax(tabuleiro, dificuldade)
                elif escolha == 2:
                    if jogadas % 2 == 0:
                        invalid_move = jogada_ia_MLP(tabuleiro, mlp)
                        if invalid_move:
                            invalid_moves += 1
                            print("Interrompendo a partida devido a jogada inválida da MLP.")
                           # mlp.update_fitness(game_result, invalid_moves)  # Reporta o tabuleiro
                            #mlp.next_chromosome()  # Avança para o próximo cromossomo
                            break  # Sai da partida atual e reinicia uma nov
                    else:
                        jogada_ia_MiniMax(tabuleiro, dificuldade)
                elif escolha == 3:
                    if jogadas % 2 == 0:
                        jogada_humana(tabuleiro)
                    else:
                        jogada_ia_MLP(tabuleiro, mlp)

                estado = verificar_estado(tabuleiro)
                if estado == 1:
                    imprimir_tabuleiro(tabuleiro)
                    print("IA Minimax venceu!")
                    vitorias_minimax += 1
                    game_result -= 1  # Penaliza o cromossomo atual
                    break
                elif estado == 2:
                    imprimir_tabuleiro(tabuleiro)
                    print("Jogador / IA MLP venceu!")
                    vitorias_mlp += 1
                    game_result = 1  # Reforça o cromossomo atual
                    break
                elif estado == 3:
                    imprimir_tabuleiro(tabuleiro)
                    print("Empate!")
                    empates += 1
                    game_result = 0  # Penaliza o cromossomo atual
                    break

                jogadas += 1

            # Atualiza a aptidão do cromossomo atual com base nas penalidades
            mlp.update_fitness(game_result, invalid_moves)
            game_result =0
            mlp.next_chromosome()  # Avança para o próximo cromossomo


        # Exibe os resultados finais
        print("\n--- Resultado Final ---")
        print(f"Vitórias Jogador / IA MLP: {vitorias_mlp}")
        print(f"Vitórias da IA Minimax: {vitorias_minimax}")
        print(f"Empates: {empates}")

        # Pergunta se deseja jogar novamente
        while True:
            resposta = input("Deseja jogar novamente? (s/n): ").lower()
            if resposta in ['s', 'n']:
                continuar_jogando = resposta == 's'
                break
            else:
                print("Por favor, digite 's' para sim ou 'n' para não.")

# Executa o jogo
jogar()
